# ARTEMIS Correlation Calculator
---
Correlate any near-Earth data with ARTEMIS around the Moon
Choose from THEMIS or MMS in-situ measurements, or OMNI data projected to the bow shock

In [197]:
# General libraries
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
import csv

# Pyspedas libraries
import pyspedas
from pytplot import del_data, get_data, get_timespan, store_data, tplot_options, tplot_names, tplot, tplot_math

In [223]:
def import_events(path, satellite): # Function to import the data from a satellite to compare with Artemis: either OMNI, THEMIS, or MMS
    probe = [] # Initialize probe ID array
    bowshock_times = [] # Initialize start and stop time array for 'satellite'
    lunar_times = [] # Initialize start and stop time array for ARTEMIS

    with open(path, newline='') as events:
        rows = csv.reader(events)
        next(rows)
        for r in rows:
            bowshock_times.append([(dt.datetime.strptime(r[0], '%Y-%m-%dT%H:%M:%S.%fZ')+dt.timedelta(minutes=30)).strftime('%Y-%m-%d/%H:%M:%S'), (dt.datetime.strptime(r[1], '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d/%H:%M:%S')])
            lunar_times.append([(dt.datetime.strptime(r[0], '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d/%H:%M:%S'), (dt.datetime.strptime(r[1], '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d/%H:%M:%S')])
            if satellite == 'mms':
                probe.append(r[2])
            elif satellite == 'themis':
                probe.append(str(r[3]))
            elif satellite == 'omni':
                probe.append('o')

        if satellite == 'omni':
            keys = np.arange(len(probe)) # Create dictionary keys 0...n equal to the number of input events
            types = ('time', 'bx', 'by', 'bz', 'vx', 'vy', 'vz', 'n', 'T') # Keys for the sub-dictionary in each event
            data = dict((i, dict.fromkeys(types)) for i in keys) # Create the 'data' dictionary
            for i in keys: # Iterate through every event
                import_data = pyspedas.omni.data(trange=bowshock_times[i], datatype='1min', level='hro2', time_clip=True) # Import the tplot variable for the ith event
                varnames = ['BX_GSE', 'BY_GSE', 'BZ_GSE', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']
                for p in varnames:
                    tplot_math.interp_nan(p)
                products = [get_data('IMF')[0], get_data('BX_GSE')[1], get_data('BY_GSE')[1], get_data('BZ_GSE')[1], get_data('Vx')[1], get_data('Vy')[1], get_data('Vz')[1], get_data('proton_density')[1], get_data('T')[1]] # Each event in the 'data' dictionary will consist of these products
                for j, k in enumerate(types): # Now go through the keys of the sub-dictionary
                    data[i][k] = products[j] # For the first sub-dict key, set that equal to the first product (and so on)...

                data[i]['time'] = data[i]['time'].astype('object')
                for n in range(len(data[i]['time'])):
                    data[i]['time'][n] = dt.datetime.utcfromtimestamp(data[i]['time'][n])


        if satellite == 'themis':
            keys = np.arange(len(probe)) # Create dictionary keys 0...n equal to the number of input events
            fgm_types = ('time', 'bx', 'by', 'bz') # Keys for the sub-dictionary in each event
            esa_types = ('time', 'vx', 'vy', 'vz', 'n', 'T')
            themis_fgm_data = dict((i, dict.fromkeys(fgm_types)) for i in keys) # Create the 'fgm_data' dictionary
            themis_esa_data = dict((i, dict.fromkeys(esa_types)) for i in keys) # Create the 'esa_data' dictionary
            for i in keys:
                themis_fgm_import = pyspedas.themis.fgm(trange=bowshock_times[i], probe=probe[i], time_clip=True, varnames='th'+probe[i]+'_fgs_gse')
                themis_esa_import = pyspedas.themis.esa(trange=bowshock_times[i], probe=probe[i], time_clip=True, varnames=['th'+probe[i]+'_peif_density', 'th'+probe[i]+'_peif_avgtemp', 'th'+probe[i]+'_peif_velocity_gse'])
                fgm_products = [get_data('th'+probe[i]+'_fgs_gse')[0], get_data('th'+probe[i]+'_fgs_gse')[1][:,0], get_data('th'+probe[i]+'_fgs_gse')[1][:,1], get_data('th'+probe[i]+'_fgs_gse')[1][:,2]]
                esa_products = [get_data('th'+probe[i]+'_peif_velocity_gse')[0], get_data('th'+probe[i]+'_peif_velocity_gse')[1][:,0], get_data('th'+probe[i]+'_peif_velocity_gse')[1][:,1], get_data('th'+probe[i]+'_peif_velocity_gse')[1][:,2], get_data('th'+probe[i]+'_peif_density')[1], get_data('th'+probe[i]+'_peif_avgtemp')[1]]
                for j, k in enumerate(fgm_types):
                    themis_fgm_data[i][k] = fgm_products[j]
                for m, n in enumerate(esa_types):
                    themis_esa_data[i][n] = esa_products[m]

        keys = np.arange(len(probe)) # Create dictionary keys 0...n equal to the number of input events
        fgm_types = ('time', 'bx', 'by', 'bz') # Keys for the sub-dictionary in each event
        esa_types = ('time', 'vx', 'vy', 'vz', 'n', 'T')
        artemis_fgm_data = dict((i, dict.fromkeys(fgm_types)) for i in keys) # Create the 'fgm_data' dictionary
        artemis_esa_data = dict((i, dict.fromkeys(esa_types)) for i in keys) # Create the 'esa_data' dictionary
        for i in keys:
            artemis_fgm_import = pyspedas.themis.fgm(trange=lunar_times[i], probe='b', time_clip=True, varnames='thb_fgs_gse')
            artemis_esa_import = pyspedas.themis.esa(trange=lunar_times[i], probe='b', time_clip=True, varnames=['thb_peif_density', 'thb_peif_avgtemp', 'thb_peif_velocity_gse'])
            fgm_products = [get_data('thb_fgs_gse')[0], get_data('thb_fgs_gse')[1][:,0], get_data('thb_fgs_gse')[1][:,1], get_data('thb_fgs_gse')[1][:,2]]
            esa_products = [get_data('thb_peif_velocity_gse')[0], get_data('thb_peif_velocity_gse')[1][:,0], get_data('thb_peif_velocity_gse')[1][:,1], get_data('thb_peif_velocity_gse')[1][:,2], get_data('thb_peif_density')[1], get_data('thb_peif_avgtemp')[1]]
            for j, k in enumerate(fgm_types):
                artemis_fgm_data[i][k] = fgm_products[j]
            for j, k in enumerate(esa_types):
                artemis_esa_data[i][k] = esa_products[j]

            artemis_fgm_data[i]['time'] = artemis_fgm_data[i]['time'].astype('object')
            artemis_esa_data[i]['time'] = artemis_esa_data[i]['time'].astype('object')
            for n in range(len(artemis_fgm_data[i]['time'])):
                artemis_fgm_data[i]['time'][n] = dt.datetime.utcfromtimestamp(artemis_fgm_data[i]['time'][n])
            for n in range(len(artemis_esa_data[i]['time'])):
                artemis_esa_data[i]['time'][n] = dt.datetime.utcfromtimestamp(artemis_esa_data[i]['time'][n])



    if satellite == 'omni':
        return data, artemis_fgm_data, artemis_esa_data
    if satellite == 'themis':
        return themis_fgm_data, themis_esa_data, artemis_fgm_data, artemis_esa_data

In [235]:
omni, a_fgm, a_esa = import_events('../eventlist/eventlist.csv', 'omni')
#t_fgm, t_esa, a_fgm, a_esa = import_events('../eventlist/eventlist.csv', 'themis')

25-Feb-23 17:39:39: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/omni/omni_cdaweb/hro2_1min/2011/
25-Feb-23 17:39:39: File is current: omni_data/hro2_1min/2011/omni_hro2_1min_20111001_v01.cdf
25-Feb-23 17:39:40: Time clip was applied to: IMF
25-Feb-23 17:39:40: Time clip was applied to: PLS
25-Feb-23 17:39:40: Time clip was applied to: IMF_PTS
25-Feb-23 17:39:40: Time clip was applied to: PLS_PTS
25-Feb-23 17:39:40: Time clip was applied to: percent_interp
25-Feb-23 17:39:40: Time clip was applied to: Timeshift
25-Feb-23 17:39:40: Time clip was applied to: RMS_Timeshift
25-Feb-23 17:39:40: Time clip was applied to: RMS_phase
25-Feb-23 17:39:40: Time clip was applied to: Time_btwn_obs
25-Feb-23 17:39:40: Time clip was applied to: F
25-Feb-23 17:39:40: Time clip was applied to: BX_GSE
25-Feb-23 17:39:40: Time clip was applied to: BY_GSE
25-Feb-23 17:39:40: Time clip was applied to: BZ_GSE
25-Feb-23 17:39:40: Time clip was applied to: BY_GSM
25-Feb-23 17:39:40: Time clip 

The next cell defines the averaging function for reducing ARTEMIS and THEMIS to a 1-minute cadence

In [227]:
def average(times, data):
    minute = times[0].minute # Set the current first minute of the data set
    timeAvgs = [] # Create empty array to store the time-averaged values
    avgArr = [] # Create empty storage array
    timeStep = [] # Create empty time step array
    for i in range(len(times)): # Index the values
        if times[i].minute == minute: # If the time of the next value equals the one set for the minute
            avgArr.append(data[i]) # Append this to the storage array
        elif times[i].minute == minute + 1: # If the time of the next value equals the next minute
            #print(avgArr)
            timeAvgs.append(np.average(avgArr)) # Average the storage array and append it to the time-averaged value array
            #print(np.average(avgArr))
            timeStep.append(dt.datetime(times[i-1].year, times[i-1].month, times[i-1].day, times[i-1].hour, times[i-1].minute, 00)) # Create a timestamp for the previous minute centered at 0s to line up with the orbit timestamps
            minute= times[i].minute # Set the new current minute to start averaging over
            avgArr = [] # Clear the storage array
        elif times[i].minute == minute - 59: # This is for rollover: when the next minute is 0
            #print(avgArr)
            timeAvgs.append(np.average(avgArr))
            #print(np.average(avgArr))
            timeStep.append(dt.datetime(times[i-1].year, times[i-1].month, times[i-1].day, times[i-1].hour, times[i-1].minute, 00))
            minute = times[i].minute
            avgArr = []
    return timeStep, timeAvgs # Return the time-averaged array and the timestamp array

In [236]:
fgm_to_avg = ('bx', 'by', 'bz')

for i in a_fgm:
    for j in fgm_to_avg:
        time_storeage, a_fgm[i][j] = average(a_fgm[i]['time'], a_fgm[i][j])
    a_fgm[i]['time'] = time_storeage